<a href="https://colab.research.google.com/github/stephenbaum/Comparing-Slopes/blob/main/comparing_slopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# import the packages of interest
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats

In [15]:
# the most simple version - you just plug everything in
def comp_slopes(b1, b2, se1, se2):
  # take the differences in slopes
  slop_dif = b1 - b2
  # square the ses
  se1_sq = se1**2
  se2_sq = se2**2
  se_comp = se1_sq + se2_sq # combine the ses
  sqrt_se_comp = math.sqrt(se_comp)
  z = round(slop_dif / sqrt_se_comp, 4) # can change if needed
  return (f"The Z statistic is {z}")

In [16]:
# testing using example one from Paternoster et al., 1998
comp_slopes(.404, .221, .094, .091)

'The Z statistic is 1.3987'

In [ ]:
# now, a more involved version with simulated data and models

In [41]:
# set a seed
np.random.seed(1234)

In [42]:
# create the id variable
id_one = np.arange(1,1001)
id_two = np.arange(1,1001)

In [43]:
# create two predictors
pred_one = np.random.randint(0, 2, size=1000)
pred_two = np.random.randint(0, 2, size=1000)

In [48]:
# create the dependent variables
dv_one = []
dv_two = []
for i in range(1000):
  if pred_one[i] == 0:
    mean = 45
  elif pred_one[i] == 1:
    mean = 55
  dv_one.append(np.random.normal(loc=mean, scale=7)) # make the sd a little different here
for i in range(1000):
  if pred_two[i] == 0:
    mean = 50
  elif pred_two[i] == 1:
    mean = 70
  dv_two.append(np.random.normal(loc=mean, scale=5))

In [49]:
# create two dfs
df_one = pd.DataFrame({'ID': id_one,
                       'Predictor': pred_one,
                       'DV': dv_one})
df_two = pd.DataFrame({'ID': id_two,
                       'Predictor': pred_two,
                       'DV': dv_two})

In [50]:
# look at the dfs
df_two.head()

,ID,Predictor,DV
0,1,1,71.412836
1,2,1,67.705344
2,3,1,70.362099
3,4,0,50.074108
4,5,1,69.965392


In [51]:
# look at some of the summary stats
group1_stats = df_one.groupby('Predictor')['DV'].agg(['mean','std']).round(2)
print(f"For Group1: \n{group1_stats.to_string(justify='right')}")
group2_stats = df_two.groupby('Predictor')['DV'].agg(['mean','std']).round(2)
print(f"For Group2: \n{group2_stats.to_string(justify='right')}")

For Group1: 
            mean   std
Predictor             
0          45.25  6.97
1          54.93  6.78
For Group2: 
            mean   std
Predictor             
0          50.14  4.97
1          70.07  5.23


In [ ]:
# change predictor variables to numeric
df_one['Predictor'] = pd.to_numeric(df_one['Predictor'])
df_two['Predictor'] = pd.to_numeric(df_two['Predictor'])

In [52]:
# the first model
mod1 = smf.ols(formula='DV ~ Predictor', data=df_one).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:                     DV   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     494.9
Date:                Fri, 28 Feb 2025   Prob (F-statistic):           2.36e-89
Time:                        04:43:06   Log-Likelihood:                -3345.5
No. Observations:                1000   AIC:                             6695.
Df Residuals:                     998   BIC:                             6705.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     45.2477      0.315    143.489      0.0

In [53]:
# the second model
mod2 = smf.ols(formula='DV ~ Predictor', data=df_two).fit()
print(mod2.summary())

                            OLS Regression Results                            
Dep. Variable:                     DV   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.792
Method:                 Least Squares   F-statistic:                     3802.
Date:                Fri, 28 Feb 2025   Prob (F-statistic):               0.00
Time:                        04:43:28   Log-Likelihood:                -3048.4
No. Observations:                1000   AIC:                             6101.
Df Residuals:                     998   BIC:                             6111.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.1376      0.234    214.438      0.0

In [ ]:
# can eyeball the coefficients beforehand, of course

In [77]:
# function to do the entire comparison just by throwing in the two models
def comp_slopes_mods(mod1, mod2):
  # get all the slopes and intercepts
  b1 = mod1.params.iloc[1]
  b2 = mod2.params.iloc[1]
  se1 = mod1.bse.iloc[1]
  se2 = mod2.bse.iloc[1]

  # now, we are in a familiar space
  slop_dif = b1 - b2
  se1_sq = se1**2
  se2_sq = se2**2
  se_comp = se1_sq + se2_sq # combine the ses
  sqrt_se_comp = math.sqrt(se_comp)
  z_value = round(slop_dif / sqrt_se_comp, 4) # can change if needed

  # get the p-value associated with the z statistic
  p_value = stats.norm.sf(abs(z_value)) * 2 # we're taking the absolute the value, can change if needed

  # can change if needed
  print(f"The slope for the first model is {b1.round(2)} and the se is {se1.round(2)}")
  print(f"The slope for the second model is {b2.round(2)} and the se is {se2.round(2)}")
  print(f"The Z statistic is {z_value.round(2)} and the p-value is {p_value}") # not rounding the p-value because it is fun
  if abs(z_value) > 1.96:
    print(f"The difference in slopes is statistically significant!")
  else:
    print(f"The difference in slopes is NOT statistically significant!")

In [78]:
# now, comparison of the two model
comp_slopes_mods(mod1, mod2)

The slope for the first model is 9.68 and the se is 0.44
The slope for the second model is 19.93 and the se is 0.32
The Z statistic is -18.91 and the p-value is 9.153685236338209e-80
The difference in slopes is statistically significant!
